# Project Deliverable 3 - Group 33
#### Group Members: Bethany Findlay, Charlotte Albert, Kaykay Akpama, Kosi Udechukwu

## Notebook Set-Up

In [10]:
#Import necessary libraries 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn import metrics
from sklearn import inspection
from sklearn.inspection import PartialDependenceDisplay
from sklearn.metrics import PredictionErrorDisplay
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_validate, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from IPython.display import display
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
#Read cleaned data from past project deliverables
df=pd.read_csv("project_deliverable_1_cleaned.csv")

In [3]:
# Same feature engineering from deliverable 2
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], errors='coerce')
df['pickup_minute'] = df['pickup_datetime'].dt.minute
df['pickup_time_fractional'] = df['pickup_hour'] + df['pickup_minute'] / 60

day_mapping = {
    'Monday': 0, 'Tuesday': 1, 'Wednesday': 2,
    'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6
}
df['pickup_day_num'] = df['pickup_day'].map(day_mapping)

## 1. Full Pipeline Construction

In [4]:
# Defining the feature set, using the expanded feature set from Deliverable 2

numeric_features = [
    'trip_distance_km',
    'pickup_day_num',
    'pickup_hour',
    'pickup_minute',
    'pickup_time_fractional',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude'
]

target = 'trip_duration'

In [5]:
# preprocessing with ColumnTransformer
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features)
    ]
)

StandardScaler puts all numeric features on a similar scale, helping models train more effectively. The ColumnTransformer is used so the preprocessing steps apply only to the selected numeric columns. This keeps the workflow clear and ensures the same steps are used during both training and testing.

In [6]:
# building 2 model pipelines (KNN Regression & Linear Regression)
linreg_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LinearRegression())
])

knn_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", KNeighborsRegressor(n_neighbors=10))
])

The two model families selected for comparison are Linear Regression and KNN Regression, similarly to deliverable 2. Linear Regression provides a simple baseline model that is fast and easy to interpret. KNN Regression captures more complex patterns by looking at nearby datapoints.

In [7]:
# train/test split
X = df[numeric_features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [8]:
# fit and evaluate with RMSE, MAE, R^2
def evaluate(model, X_test, y_test):
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds)
    return mae, rmse, r2

linreg_model.fit(X_train, y_train)
knn_model.fit(X_train, y_train)

linreg_metrics = evaluate(linreg_model, X_test, y_test)
knn_metrics = evaluate(knn_model, X_test, y_test)

The evaluation function uses MAE, RMSE, and R^2 because these three metrics give an overall view of the model performance for a regression task. MAE shows the average size of errors, RMSE penalizes larger mistakes more strongly, and R^2 shows how much the variation in trip duration the model can explain. Multiple metrics helps to compare multiple models against one another.

In [9]:
# print results for comparison
print("Linear Regression Results:")
print(f"MAE: {linreg_metrics[0]:.3f} secs")
print(f"RMSE: {linreg_metrics[1]:.3f} secs")
print(f"R^2: {linreg_metrics[2]:.3f}\n")

print("KNN Regression Results:")
print(f"MAE: {knn_metrics[0]:.3f} secs")
print(f"RMSE: {knn_metrics[1]:.3f} secs")
print(f"R^2: {knn_metrics[2]:.3f}")

Linear Regression Results:
MAE: 276.784 secs
RMSE: 407.309 secs
R^2: 0.604

KNN Regression Results:
MAE: 199.656 secs
RMSE: 316.123 secs
R^2: 0.761


## 2. Systematic Hyperparameter Tuning

applying pca

In [27]:
knn_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ('pca', PCA()),
    ("knn", KNeighborsRegressor(n_neighbors = 10))
])

In [28]:
pca_param_grid = {
    "pca__n_components": [2, 4, 6, 8],
}

scoring = {
    "r2": "r2",
    "neg_mean_absolute_error": "neg_mean_absolute_error",
    "neg_root_mean_squared_error": "neg_root_mean_squared_error"
}

In [29]:
knn_tuning_pca = GridSearchCV(knn_pipeline, pca_param_grid, scoring = scoring, refit = "r2", cv=5, n_jobs=-1)
knn_tuning_pca.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('knn',
                                        KNeighborsRegressor(n_neighbors=10))]),
             n_jobs=-1, param_grid={'pca__n_components': [2, 4, 6, 8]},
             refit='r2',
             scoring={'neg_mean_absolute_error': 'neg_mean_absolute_error',
                      'neg_root_mean_squared_error': 'neg_root_mean_squared_error',
                      'r2': 'r2'})

In [ ]:
results_pca = pd.DataFrame(knn_tuning_pca.cv_results_)
show = results_pca[[
    'param_pca__n_components',
    'mean_test_r2',
    'mean_test_neg_mean_absolute_error',
    'mean_test_neg_root_mean_squared_error'
]]

print(show)

Testing number of neighbors

In [ ]:
knn_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsRegressor())
])

In [ ]:
param_grid = {
    "knn__n_neighbors": [7, 10, 13, 15, 17, 19]
}

#"pca__n_components": [3, 5, 7, 9],

scoring = {
    "r2": "r2",
    "neg_mean_absolute_error": "neg_mean_absolute_error",
    "neg_root_mean_squared_error": "neg_root_mean_squared_error"
}

In [25]:
knn_tuning = GridSearchCV(knn_pipeline, param_grid, scoring = scoring, refit = "r2", cv=5, n_jobs=-1)
knn_tuning.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('knn', KNeighborsRegressor())]),
             n_jobs=-1,
             param_grid={'knn__n_neighbors': [7, 10, 13, 15, 17, 19]},
             refit='r2',
             scoring={'neg_mean_absolute_error': 'neg_mean_absolute_error',
                      'neg_root_mean_squared_error': 'neg_root_mean_squared_error',
                      'r2': 'r2'})

In [26]:
results = pd.DataFrame(knn_tuning.cv_results_)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(results)

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       7.965645      1.166811        94.680955        4.670848   
1       7.578070      1.658168       106.146655        5.734487   
2       5.819586      0.040470       124.744490        5.396086   
3       6.751458      0.652680       132.760827        3.559967   
4       6.654175      0.660581       133.881684        6.338935   
5       6.202133      0.665584       123.514582        7.809998   

   param_knn__n_neighbors                    params  split0_test_r2  \
0                       7   {'knn__n_neighbors': 7}        0.753415   
1                      10  {'knn__n_neighbors': 10}        0.760142   
2                      13  {'knn__n_neighbors': 13}        0.762774   
3                      15  {'knn__n_neighbors': 15}        0.763674   
4                      17  {'knn__n_neighbors': 17}        0.764033   
5                      19  {'knn__n_neighbors': 19}        0.764126   

   split1_test_r2  split2_test_r

## 3. Final Model Selection and Evaluation

## 4. Integration of TA Feedback

## 5. Final Reflection

## 6. Individual Contributions

## BONUS: Kaggle Submission